In [1]:
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import os
import collections
import numpy as np
import pandas as pd
from tqdm import tqdm, tqdm_notebook
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.layers import Input, Dense, Activation, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalAveragePooling2D
from keras.models import Model
from keras.applications import VGG16
from keras.optimizers import Adam
from tensorflow import keras
import random
from PIL import Image

In [2]:
model_Mid = keras.models.load_model("D:\\Project Himachal\\Data\ForCNN\\Takrimodel")
model_Top = keras.models.load_model("D:\\Project Himachal\\Data\ForCNN\\TakrimodelTop")
master_label_Mid = ['100',  '101',  '102',  '103',  '105',  '106',  '107',  '108',  '110',  '111',  '112',  '113',  '114',  '115',  '116',  '117',  '118',  '119',  '120',  '121',  '123',  '124',  '125',  '126',  '127',  '128',  '130',  '131',  '200',  '201', '202',  '203',  '204',  '205',  '206',  '302',  '303',  '402',  '403',  '404',  '405',  '406',  '408',  '409',  '410',  '500']
master_label_Top = ['301',  '302',  '306',  '307',  '308',  '309',  '310']
dict_ = {}
test_df = pd.read_csv('D:\\Project Himachal\\Data\\LabelMapper.csv')
for row in test_df.iterrows():
    label = (row[1][0])
    unicode = (row[1][1])
    dict_[label] = unicode

In [3]:
def isEmpty(img_path):
    image = cv2.imread(img_path,cv2.IMREAD_GRAYSCALE) # reading the image
    
    hieght = image.shape[0]
    width = image.shape[1]    
    
    for y_axis in range (hieght):        
        for x_axis in range (width):
            if (image[y_axis,x_axis]!=0):            
                return 0           
    return 1

def findCharacterCNN(imges_test,location):
    #imges_test = "D:\\Project Himachal\\Data\\Temp\\Temp_Resized_CharPart.png"
    if(location=="M"):
        model = model_Mid
        master_label = master_label_Mid
    if (location=="T"):
        model = model_Top
        master_label = master_label_Top
    X_tst = []
    Test_imgs = []
    
    temp = cv2.imread(imges_test)
    temp = cv2.cvtColor(temp, cv2.COLOR_BGR2GRAY)
    temp = np.expand_dims(temp, axis=2)
    X_tst.append(temp)      
    Test_imgs.append(imges_test)
    X_tst = np.asarray(X_tst)
    X_tst = X_tst.astype('float32')
    X_tst /= 255
    
    test_predictions = model.predict(X_tst)
    for answer in test_predictions:
        index = -1
        max_index = -1
        tc = 0 
        for c,v in enumerate(answer):
            if v>max_index:
                max_index = v
                index = c
            tc = tc + v
            
    label = int(master_label[index])
    #print(label)
    #print(dict_)
    return dict_[label]

def find_hieght(img_path):
    image = cv2.imread(img_path) # reading the image    
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) # convert2grayscale
    (thresh, binary) = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU) # convert2binary
    
    hieght = binary.shape[0]
    width = binary.shape[1]
 
    ht = 0
    
    for y_axis in range (hieght):
        black = 0
        for x_axis in range (width):
            if (binary[y_axis,x_axis]==0):            
                black = black+1
            #print(binary[y_axis,x_axis])
        #print(black) 
        if (black>0):
            ht = ht+1
    
    return ht

In [4]:
def find_biggestbox(binary):
    hieght = binary.shape[0]
    width = binary.shape[1]
    buffer = 2
    x_start=0
    found=0
    for x_axis in range (width):
        black = 0
        for y_axis in range (hieght):
            if (binary[y_axis,x_axis]==0):            
                black = black+1
            #print(binary[y_axis,x_axis])
        #print(black) 
        if (black==0) and (found==0):
            x_start = x_axis
        else:
            found=1
    x_start = x_start - buffer
    if (x_start<0):
        x_start = 0
    
    x_end=0
    found=0
    for x_axis in range (width):
        black = 0
        x_axis = width - x_axis-1
        for y_axis in range (hieght):
            if (binary[y_axis,x_axis]==0):            
                black = black+1
            #print(binary[y_axis,x_axis])
        #print(black) 
        if (black==0) and (found==0):
            x_end = x_axis
        else:
            found=1
    if(x_end==0):
        x_end = width
    x_end = x_end + buffer
    if (x_end>width):
        x_end = width
            
    y_start=0
    found=0
    for y_axis in range (hieght):
        black = 0
        for x_axis in range (width):
            if (binary[y_axis,x_axis]==0):            
                black = black+1
            #print(binary[y_axis,x_axis])
        #print(black) 
        if (black==0) and (found==0):
            y_start = y_axis
        else:
            found=1
    y_start = y_start - buffer
    if (y_start<0):
        y_start = 0

            
    y_end=0
    found=0
    for y_axis in range (hieght):
        black = 0
        y_axis = hieght - y_axis-1
        for x_axis in range (width):
            if (binary[y_axis,x_axis]==0):            
                black = black+1
            #print(binary[y_axis,x_axis])
        #print(black) 
        if (black==0) and (found==0):
            y_end = y_axis
        else:
            found=1
    if(y_end==0):
        y_end = hieght
    y_end = y_end + buffer
    if (y_end>hieght):
        y_end = hieght
 

    return (x_start,x_end,y_start,y_end)   

In [5]:
def extract_Character_Full(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) # convert2grayscale
    (thresh, binary) = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU) # convert2binary
    #print(binary.shape)
    hieght = binary.shape[0]
    width = binary.shape[1]
    check = 0
    x1 = []
    y1 = []
    for x_axis in range (width):
        black = 0
        for y_axis in range (hieght):
            if (binary[y_axis,x_axis]==0):
                weight = abs(y_axis - (hieght/2))/(hieght/2)
                if(y_axis>hieght*.7):
                    weight = weight*.4
                black = abs(black + 1 - weight)
            #print(binary[y_axis,x_axis])
        #print(black) 
        x1.append(x_axis)
        y1.append(black/hieght)

    length_ = len(y1)
    minima = []
    for index in range(length_):
        range_ = 5
        flag = 1
        for internal_loop in range (range_):
            if (index-internal_loop)>=0 and y1[index-internal_loop]<y1[index]:
                flag = 0
            if (index+internal_loop)<length_ and y1[index+internal_loop]<y1[index]:
                flag = 0
        if (flag==1) and y1[index]<.04:   
            #print(y1[index])
            minima.append(index)

    for num in minima:
        for a in range(1,5):
            if (num+a) in minima:
                minima.remove(num+a)
            else:
                break

    if 0 not in (minima):
        minima.append(0)

    if width not in (minima):
        minima.append(width)
    
    minima_v = minima

    #Now lets divide horizontally
    x1 = []
    y1 = []
    for y_axis in range (hieght):
        black = 0
        for x_axis in range (width):
            if (binary[y_axis,x_axis]==0):
                weight = abs(x_axis - (width/2))/(width/2)
                black = abs(black + 1 - weight)
                #black = black + 1
            #print(binary[y_axis,x_axis])
        x1.append(y_axis)
        y1.append(black/width) 

    length_ = len(y1)
    minima = []
    for index in range(length_):
        range_ = 8
        flag = 1
        for internal_loop in range (range_):
            if (index-internal_loop)>=0 and y1[index-internal_loop]<y1[index]:
                flag = 0
            if (index+internal_loop)<length_ and y1[index+internal_loop]<y1[index]:
                flag = 0
        if (flag==1) or y1[index]<.2:
            minima.append(index)

    for num in minima:
        for a in range(1,10):
            if (num>hieght/2):
                check = num+a
            if (num<hieght/2):
                check = num-a
            if (check) in minima:
                minima.remove(check)
            else:
                break
    top_= 0
    bottom_ = hieght
    for number in minima:
        if (number > top_) and number < hieght/3:
            top_ = number
        if (number < bottom_) and (hieght-number) < hieght/3:
            bottom_ = number
    
    w = img.shape[1]
    h = img.shape[0]

    out_W=[]
    
    prev = 0
    for ind_val in minima_v:        
        if ind_val == 0:
            continue
        #print(0,top_,prev,ind_val)
        #print(top_,bottom_,prev,ind_val)
        #print(bottom_,h,prev,ind_val)
        roi0 = img[0:h,prev:ind_val]        
        out_W.append(roi0)
        prev = ind_val

    return out_W

In [6]:
def removeBottomLine(img_path):
    image = cv2.imread(img_path,cv2.IMREAD_GRAYSCALE) # reading the image
    
    hieght = image.shape[0]
    width = image.shape[1]
    
    img = np.zeros(shape=(hieght, width))
    
    val = 0
    flag = 0
    for y_axis in range (hieght):        
        val = 0
        for x_axis in range (width):
            if (image[y_axis,x_axis]==0):            
                val = val + 1                
        #print(val)
        if (val>22)or(flag==1): 
            flag = 1
            for y_axis_delta in range(2):
                for x_axis in range (width):
                    img[y_axis-y_axis_delta,x_axis] = 255                
        else:
            for x_axis in range (width):
                img[y_axis,x_axis] = image[y_axis,x_axis]
        if (val==0):
            flag=0
    return img

def make_binary(Img_Original):
    h,b = (Img_Original.shape)

    binary_img = np.zeros([h, b])

    for col in range(h):
        for row in range(b):
            if (Img_Original[col][row])>=200:
                binary_img[col][row]= 0
            else:
                binary_img[col][row]= 1

    return binary_img

def invert(Img_Original):
    h,b = (Img_Original.shape)

    binary_img = np.zeros([h, b])

    for col in range(h):
        for row in range(b):
            if (Img_Original[col][row])==0:
                binary_img[col][row]= 255
            else:
                binary_img[col][row]= 0

    return binary_img

def neighbours(x,y,image):
    "Return 8-neighbours of image point P1(x,y), in a clockwise order"
    img = image
    x_1, y_1, x1, y1 = x-1, y-1, x+1, y+1
    return [ img[x_1][y], img[x_1][y1], img[x][y1], img[x1][y1],     # P2,P3,P4,P5
                img[x1][y], img[x1][y_1], img[x][y_1], img[x_1][y_1] ]    # P6,P7,P8,P9

def transitions(neighbours):
    "No. of 0,1 patterns (transitions from 0 to 1) in the ordered sequence"
    n = neighbours + neighbours[0:1]      # P2, P3, ... , P8, P9, P2
    return sum( (n1, n2) == (0, 1) for n1, n2 in zip(n, n[1:]) )  # (P2,P3), (P3,P4), ... , (P8,P9), (P9,P2)

def thinning(image):
    "the Zhang-Suen Thinning Algorithm"
    Image_Thinned = image.copy()  # deepcopy to protect the original image
    changing1 = changing2 = 1        #  the points to be removed (set as 0)
    while changing1 or changing2:   #  iterates until no further changes occur in the image
        # Step 1
        changing1 = []
        rows, columns = Image_Thinned.shape               # x for rows, y for columns
        for x in range(1, rows - 1):                     # No. of  rows
            for y in range(1, columns - 1):            # No. of columns
                P2,P3,P4,P5,P6,P7,P8,P9 = n = neighbours(x, y, Image_Thinned)
                if (Image_Thinned[x][y] == 1     and    # Condition 0: Point P1 in the object regions 
                    2 <= sum(n) <= 6   and    # Condition 1: 2<= N(P1) <= 6
                    transitions(n) == 1 and    # Condition 2: S(P1)=1  
                    P2 * P4 * P6 == 0  and    # Condition 3   
                    P4 * P6 * P8 == 0):         # Condition 4
                    changing1.append((x,y))
        for x, y in changing1: 
            Image_Thinned[x][y] = 0
        # Step 2
        changing2 = []
        for x in range(1, rows - 1):
            for y in range(1, columns - 1):
                P2,P3,P4,P5,P6,P7,P8,P9 = n = neighbours(x, y, Image_Thinned)
                if (Image_Thinned[x][y] == 1   and        # Condition 0
                    2 <= sum(n) <= 6  and       # Condition 1
                    transitions(n) == 1 and      # Condition 2
                    P2 * P4 * P8 == 0 and       # Condition 3
                    P2 * P6 * P8 == 0):            # Condition 4
                    changing2.append((x,y))    
        for x, y in changing2: 
            Image_Thinned[x][y] = 0    
    return Image_Thinned
 
def imshow_components(labels):
    # Map component labels to hue val
    label_hue = np.uint8(179*labels/np.max(labels))
    blank_ch = 255*np.ones_like(label_hue)
    labeled_img = cv2.merge([label_hue, blank_ch, blank_ch])

    # cvt to BGR for display
    labeled_img = cv2.cvtColor(labeled_img, cv2.COLOR_HSV2BGR)

    # set bg label to black
    labeled_img[label_hue==0] = 0

    cv2.imshow('labeled.png', labeled_img)
    cv2.waitKey()
    
def imsave_components(labels):
    # Map component labels to hue val
    label_hue = np.uint8(179*labels/np.max(labels))
    blank_ch = 255*np.ones_like(label_hue)
    labeled_img = cv2.merge([label_hue, blank_ch, blank_ch])

    # cvt to BGR for display
    labeled_img = cv2.cvtColor(labeled_img, cv2.COLOR_HSV2BGR)

    # set bg label to black
    labeled_img[label_hue==0] = 0

    cv2.imshow('labeled.png', labeled_img)

def find_index(arr):
    index = 0
    row = 0
    flag=0
    for l in arr:
        #print(l,np.sum(l))
        if (np.sum(l)==0) and (flag==0):
            index=row
        else:
            flag = 1

        row=row+1
    return index

def keep_max(labels_im):
    b = labels_im.flatten()
    counter = collections.Counter(b)
    input_arr1 = np.copy(labels_im)
    
    max = 0
    for c in counter:
        if c==0:
            continue
        if (counter[c]>max):
            out = c
            max = counter[c]
    input_arr1[input_arr1!=out] = 0
    input_arr1[input_arr1==out] = 1
    return input_arr1

def keep_max2(labels_im):
    hieght = labels_im.shape[0]
    width = labels_im.shape[1]
    
    singular = []
    for y_axis in range (hieght):
        val = 0
        for x_axis in range (width):
            if (labels_im[y_axis,x_axis]!=0):            
                val = labels_im[y_axis,x_axis]
        
        singular.append(val)

    #b = singular.flatten()
    counter = collections.Counter(singular)
    input_arr1 = np.copy(labels_im)
    
    max = 0
    out = 0
    for c in counter:
        if c==0:
            continue
        if (counter[c]>max):
            out = c
            max = counter[c]
    
    input_arr1[input_arr1!=out] = 0
    input_arr1[input_arr1==out] = 1
    return input_arr1

def filled_ratio(img_path):
    image = cv2.imread(img_path) # reading the image
    image_copy = cv2.imread(img_path) # reading the image
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) # convert2grayscale
    (thresh, binary) = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU) # convert2binary

    hieght = binary.shape[0]
    width = binary.shape[1]
    
    val = 0
    for y_axis in range (hieght):        
        for x_axis in range (width):
            if (binary[y_axis,x_axis]!=0):            
                val = val + 1
    
    return val/(hieght*width)

In [7]:
def makeSameSize(img_path,savePath):
    img_b = Image.open("D:\\Project Himachal\\Data\\Labelled\\background.png")
    img_o = Image.open(img_path)
    img = cv2.imread(img_path)            
    w = (img.shape[1])
    h = (img.shape[0])
    img_b.paste(img_o, (int((50-w)/2),int((60-h)/2)), mask = img_o)
    img_b.save(savePath)

In [8]:

def divide_page_into_lines(img_path):
    image = cv2.imread(img_path) # reading the image
    image_copy = cv2.imread(img_path) # reading the image
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) # convert2grayscale
    (thresh, binary) = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU) # convert2binary
    #cv2.imshow('binary', binary)
    cv2.imwrite('D:\\Project Himachal\\Data\\Temp\\Page_binary.png', binary)
    hieght = binary.shape[0]
    width = binary.shape[1]
    
    x1 = []
    y1 = []
    whites = []
    whites_copy = []
    for y_axis in range (hieght):
        black = 0
        for x_axis in range (width):
            if (binary[y_axis,x_axis]==0):            
                black = black+1
            #print(binary[y_axis,x_axis])
        #print(black) 
        x1.append(y_axis)
        y1.append(black)
        if(black>0):
            whites.append(y_axis)
            
    
    for white in whites:
        whites_copy.append(white)
        if ((white+1) not in whites) and ((white+2) in whites):
            whites_copy.append(white+1)
    
    lineBreaks = []
    st = -1
    en = -1
    for white in whites_copy:
        if ((white+1) not in whites_copy) and ((white+2) not in whites_copy):
            en = white
        if ((white-1) not in whites_copy):
            st = white
        if(st!=-1)and(en!=-1):
            if(st!=en):
                lineBreaks.append([st,en])
            st=-1
            en=-1
    
    #print(whites)
    #print(lineBreaks)
 
   
    finalOutput = []
    
    for c,b in enumerate(lineBreaks[1:]):       
        cv2.line(image_copy, (0, b[0]), (width, b[0]), (255, 0, 0), thickness=2)
        cv2.line(image_copy, (0, b[1]), (width, b[1]), (0, 255, 0), thickness=2)
        if (abs(b[0]-b[1]))<10:
            continue
        roi=image[b[0]:b[1],0:width]
        if(abs(b[0]-b[1])>80):            
            nn = int(b[0] + abs(b[0]-b[1])/2)
            roi1 = image[b[0]:nn,0:width]
            roi2 = image[nn:b[1],0:width]
            cv2.line(image_copy, (0, nn), (width, nn), (0, 0, 255), thickness=2)
            cv2.imwrite('D:\\Project Himachal\\Data\Documents 2\\lines\\Page_' + str(img_path[-6:-4]) + '_Line_' + str(c) + '.png', roi1)
            cv2.imwrite('D:\\Project Himachal\\Data\Documents 2\\lines\\Page_' + str(img_path[-6:-4]) + '_Line_' + str(c) + '.png', roi2)
        else:
            cv2.imwrite('D:\\Project Himachal\\Data\Documents 2\\lines\\Page_' + str(img_path[-6:-4]) + '_Line_' + str(c) + '.png', roi)
            
        
       
    cv2.imwrite('D:\\Project Himachal\\Data\Documents 2\\lines\\Toto.png', image_copy)

    return finalOutput

In [9]:
img = 'D:\\Project Himachal\\Data\Documents 2\\NC\\04.jpg'
divide_page_into_lines(img)

[]

In [10]:
path = "D:\\Project Himachal\\Data\Documents 2\\NC\\"
for (root, dirs, file) in os.walk(path):
    for f in file:
        print(path+f)
        divide_page_into_lines(path+f)

D:\Project Himachal\Data\Documents 2\NC\04.jpg
D:\Project Himachal\Data\Documents 2\NC\06.jpg
D:\Project Himachal\Data\Documents 2\NC\07.jpg
D:\Project Himachal\Data\Documents 2\NC\08.jpg
D:\Project Himachal\Data\Documents 2\NC\09.jpg
D:\Project Himachal\Data\Documents 2\NC\100.jpg
D:\Project Himachal\Data\Documents 2\NC\101.jpg
D:\Project Himachal\Data\Documents 2\NC\102.jpg
D:\Project Himachal\Data\Documents 2\NC\104.jpg
D:\Project Himachal\Data\Documents 2\NC\105.jpg
D:\Project Himachal\Data\Documents 2\NC\106.jpg
D:\Project Himachal\Data\Documents 2\NC\107.jpg
D:\Project Himachal\Data\Documents 2\NC\108.jpg
D:\Project Himachal\Data\Documents 2\NC\11.jpg
D:\Project Himachal\Data\Documents 2\NC\110.jpg
D:\Project Himachal\Data\Documents 2\NC\111.jpg
D:\Project Himachal\Data\Documents 2\NC\112.jpg
D:\Project Himachal\Data\Documents 2\NC\114.jpg
D:\Project Himachal\Data\Documents 2\NC\115.jpg
D:\Project Himachal\Data\Documents 2\NC\116.jpg
D:\Project Himachal\Data\Documents 2\NC\117.jp